##### TODOs

- [ ] Add matplotlib to visualize images

# Image Classification with MNIST dataset

## Prepare environment

In [2]:
# Install dependencies
! pip install torch torchvision

    40% |█████████████                   | 211.7MB 6.6MB/s eta 0:00:4781 day, 23:58:54                      | 2.2MB 5.8MB/s eta 0:01:30    1% |▌                               | 7.9MB 6.5MB/s eta 0:01:20    2% |▉                               | 13.5MB 6.1MB/s eta 0:01:23    2% |▉                               | 13.9MB 6.0MB/s eta 0:01:25    2% |█                               | 14.9MB 6.1MB/s eta 0:01:23    3% |█                               | 15.6MB 6.8MB/s eta 0:01:15    3% |█                               | 17.0MB 7.1MB/s eta 0:01:11    3% |█▎                              | 20.3MB 4.2MB/s eta 0:01:59    4% |█▎                              | 21.0MB 6.4MB/s eta 0:01:18    4% |█▎                              | 21.3MB 6.8MB/s eta 0:01:13    5% |█▊                              | 27.1MB 5.6MB/s eta 0:01:28    6% |██                              | 31.7MB 6.4MB/s eta 0:01:16    6% |██                              | 32.0MB 6.9MB/s eta 0:01:11    6% |██                              | 32.7MB 6

    100% |████████████████████████████████| 13.9MB 3.2MB/s ta 0:00:011    90% |█████████████████████████████   | 12.6MB 6.1MB/s eta 0:00:01
    100% |████████████████████████████████| 2.0MB 7.4MB/s ta 0:00:011


In [72]:
# Import modules
import random

from math import floor

import torch

from torch.utils.data import DataLoader
from torch.utils.data import random_split

from torch import nn
from torch.nn import functional as F
from torch import optim

from torchvision import datasets
from torchvision import transforms
from torchvision import models

import numpy as np

## Download and pre-process data

TODO: Normalize data

In [64]:
# Download and load MNIST dataset
mnist_train = datasets.MNIST(root='/tmp', download=True, train=True, transform=transforms.ToTensor())
mnist_test = datasets.MNIST(root='/tmp', download=True, train=False, transform=transforms.ToTensor())

# Split the MNIST train set into train and validation subsets
num_examples = len(mnist_train)
num_validation_examples = floor(0.3*len(mnist_train))
num_train_examples = len(mnist_train) - validation_examples
print(f'Number of examples: {num_examples} \n'
      f'Number of train examples: {num_train_examples} \n'
      f'Number of validation examples: {num_validation_examples}')

sets = random_split(mnist_train, [num_train_examples, num_validation_examples])
train_set = sets[0]
validation_set = sets[1]

# Generate DataLoaders for each subset
train_loader = DataLoader(train_set, batch_size=32, shuffle=True)
validation_loader = DataLoader(validation_set, batch_size=32, shuffle=False)
test_loader = DataLoader(mnist_test, batch_size=32, shuffle=False)

Number of examples: 60000 
Number of train examples: 42000 
Number of validation examples: 18000


## Classifier 1 - BasicCNN

In [171]:
class BasicCNN(nn.Module):
    def __init__(self):
        super(BasicCNN, self).__init__()
        self.conv1 = nn.Conv2d(in_channels=1, out_channels=32, kernel_size=(3, 3))
        self.conv2 = nn.Conv2d(in_channels=32, out_channels=64, kernel_size=(2, 2))
        self.fc1 = nn.Linear(64*12*12, 32)
        self.fc2 = nn.Linear(32, 10)
        
    def forward(self, x):
        x = F.relu(F.max_pool2d(self.conv1(x), kernel_size=(2, 2)))
        # cnn output size = (input_size - kernel_size + 2*padding)/stride + 1
        # feature map size = [ (28-3+2*0)/(1) + 1 = 26 ] / 2 = 13
        x = F.relu(self.conv2(x))
        # feature map size = (13-2)/1 + 1 = 12
        x = x.view(-1, 64*12*12)
        x = F.relu(self.fc1(x))
        x = self.fc2(x)
        return F.log_softmax(x, dim=1)

## Train and evaluate model

In [184]:
# Train model
basic_cnn = BasicCNN()
criteria = nn.CrossEntropyLoss()
optimizer = optim.RMSprop(basic_cnn.parameters())

# For each epoch...
for epoch in range(10):
    # Training part
    basic_cnn.train()
    # For each mini-batch
    for batch_idx, (data, target) in enumerate(train_loader):
        # Forward
        output = basic_cnn(data)
        # Calculate loss
        loss = criteria(output, target)
        # Clear previous gradients (make gradients zero)
        optimizer.zero_grad()
        # Backpropagation
        loss.backward()
        # Update model parameters
        optimizer.step()
        
        print(f'Epoch: {epoch} [{batch_idx * len(data)}/{len(train_loader.dataset)}'
              f'({round(100. * batch_idx / len(train_loader), 2)}%)]\t'
              f'Loss: {round(loss.item(), 6)}')
    
    # Evaluation part-> Validation set
    basic_cnn.eval()
    validation_loss = 0
    correct = 0
    with torch.no_grad():
        for data, target in validation_loader:
            output = basic_cnn(data)
            validation_loss += criteria(output, target).item() # sum up batch loss
            pred = output.max(1, keepdim=True)[1] # get the index of the max log-probability
            correct += pred.eq(target.view_as(pred)).sum().item()

    validation_loss /= len(validation_loader.dataset)
    print('\n'
          f'Validation set: Average loss: {round(validation_loss, 6)}, '
          f'Accuracy: {correct}/{len(validation_loader.dataset)} '
          f'({round(100. * correct / len(validation_loader.dataset), 3)}%)\n')    

Epoch: 0 [0/42000(0.0%)]	Loss: 2.305647
Epoch: 0 [32/42000(0.08%)]	Loss: 363.421844
Epoch: 0 [64/42000(0.15%)]	Loss: 6.900511
Epoch: 0 [96/42000(0.23%)]	Loss: 2.362942
Epoch: 0 [128/42000(0.3%)]	Loss: 2.352865
Epoch: 0 [160/42000(0.38%)]	Loss: 2.287915
Epoch: 0 [192/42000(0.46%)]	Loss: 2.284725
Epoch: 0 [224/42000(0.53%)]	Loss: 2.325075
Epoch: 0 [256/42000(0.61%)]	Loss: 2.261855
Epoch: 0 [288/42000(0.69%)]	Loss: 2.310276
Epoch: 0 [320/42000(0.76%)]	Loss: 2.331645
Epoch: 0 [352/42000(0.84%)]	Loss: 2.307394
Epoch: 0 [384/42000(0.91%)]	Loss: 2.275899
Epoch: 0 [416/42000(0.99%)]	Loss: 2.346832
Epoch: 0 [448/42000(1.07%)]	Loss: 2.305277
Epoch: 0 [480/42000(1.14%)]	Loss: 2.288008
Epoch: 0 [512/42000(1.22%)]	Loss: 2.321583
Epoch: 0 [544/42000(1.29%)]	Loss: 2.307644
Epoch: 0 [576/42000(1.37%)]	Loss: 2.310762
Epoch: 0 [608/42000(1.45%)]	Loss: 2.336188
Epoch: 0 [640/42000(1.52%)]	Loss: 2.312522
Epoch: 0 [672/42000(1.6%)]	Loss: 2.31606
Epoch: 0 [704/42000(1.68%)]	Loss: 2.296219
Epoch: 0 [736/4200

Epoch: 0 [6176/42000(14.7%)]	Loss: 0.570981
Epoch: 0 [6208/42000(14.78%)]	Loss: 1.003916
Epoch: 0 [6240/42000(14.85%)]	Loss: 0.26398
Epoch: 0 [6272/42000(14.93%)]	Loss: 0.293307
Epoch: 0 [6304/42000(15.0%)]	Loss: 0.39838
Epoch: 0 [6336/42000(15.08%)]	Loss: 0.444228
Epoch: 0 [6368/42000(15.16%)]	Loss: 0.308506
Epoch: 0 [6400/42000(15.23%)]	Loss: 0.289697
Epoch: 0 [6432/42000(15.31%)]	Loss: 0.196506
Epoch: 0 [6464/42000(15.38%)]	Loss: 0.736652
Epoch: 0 [6496/42000(15.46%)]	Loss: 0.560174
Epoch: 0 [6528/42000(15.54%)]	Loss: 0.434025
Epoch: 0 [6560/42000(15.61%)]	Loss: 0.319952
Epoch: 0 [6592/42000(15.69%)]	Loss: 0.523069
Epoch: 0 [6624/42000(15.77%)]	Loss: 0.702684
Epoch: 0 [6656/42000(15.84%)]	Loss: 0.295607
Epoch: 0 [6688/42000(15.92%)]	Loss: 0.083601
Epoch: 0 [6720/42000(15.99%)]	Loss: 0.093426
Epoch: 0 [6752/42000(16.07%)]	Loss: 0.180334
Epoch: 0 [6784/42000(16.15%)]	Loss: 0.324929
Epoch: 0 [6816/42000(16.22%)]	Loss: 0.145008
Epoch: 0 [6848/42000(16.3%)]	Loss: 0.345719
Epoch: 0 [6880/

Epoch: 0 [12192/42000(29.02%)]	Loss: 0.246455
Epoch: 0 [12224/42000(29.09%)]	Loss: 0.791995
Epoch: 0 [12256/42000(29.17%)]	Loss: 0.322708
Epoch: 0 [12288/42000(29.25%)]	Loss: 0.259354
Epoch: 0 [12320/42000(29.32%)]	Loss: 0.223042
Epoch: 0 [12352/42000(29.4%)]	Loss: 0.272323
Epoch: 0 [12384/42000(29.47%)]	Loss: 0.106574
Epoch: 0 [12416/42000(29.55%)]	Loss: 0.455784
Epoch: 0 [12448/42000(29.63%)]	Loss: 0.210983
Epoch: 0 [12480/42000(29.7%)]	Loss: 0.190626
Epoch: 0 [12512/42000(29.78%)]	Loss: 0.107157
Epoch: 0 [12544/42000(29.86%)]	Loss: 0.430151
Epoch: 0 [12576/42000(29.93%)]	Loss: 0.190466
Epoch: 0 [12608/42000(30.01%)]	Loss: 0.185408
Epoch: 0 [12640/42000(30.08%)]	Loss: 0.104678
Epoch: 0 [12672/42000(30.16%)]	Loss: 0.512863
Epoch: 0 [12704/42000(30.24%)]	Loss: 0.312243
Epoch: 0 [12736/42000(30.31%)]	Loss: 0.424755
Epoch: 0 [12768/42000(30.39%)]	Loss: 0.40004
Epoch: 0 [12800/42000(30.46%)]	Loss: 0.106738
Epoch: 0 [12832/42000(30.54%)]	Loss: 0.21214
Epoch: 0 [12864/42000(30.62%)]	Loss: 0

Epoch: 0 [17984/42000(42.8%)]	Loss: 0.606319
Epoch: 0 [18016/42000(42.88%)]	Loss: 0.702849
Epoch: 0 [18048/42000(42.96%)]	Loss: 0.238927
Epoch: 0 [18080/42000(43.03%)]	Loss: 0.303874
Epoch: 0 [18112/42000(43.11%)]	Loss: 0.250006
Epoch: 0 [18144/42000(43.18%)]	Loss: 0.204191
Epoch: 0 [18176/42000(43.26%)]	Loss: 0.037402
Epoch: 0 [18208/42000(43.34%)]	Loss: 0.227175
Epoch: 0 [18240/42000(43.41%)]	Loss: 0.046238
Epoch: 0 [18272/42000(43.49%)]	Loss: 0.250893
Epoch: 0 [18304/42000(43.56%)]	Loss: 0.043958
Epoch: 0 [18336/42000(43.64%)]	Loss: 0.077806
Epoch: 0 [18368/42000(43.72%)]	Loss: 0.210274
Epoch: 0 [18400/42000(43.79%)]	Loss: 0.091754
Epoch: 0 [18432/42000(43.87%)]	Loss: 0.126204
Epoch: 0 [18464/42000(43.95%)]	Loss: 0.151016
Epoch: 0 [18496/42000(44.02%)]	Loss: 0.151462
Epoch: 0 [18528/42000(44.1%)]	Loss: 0.085628
Epoch: 0 [18560/42000(44.17%)]	Loss: 0.255017
Epoch: 0 [18592/42000(44.25%)]	Loss: 0.166379
Epoch: 0 [18624/42000(44.33%)]	Loss: 0.123329
Epoch: 0 [18656/42000(44.4%)]	Loss: 

Epoch: 0 [23936/42000(56.97%)]	Loss: 0.167143
Epoch: 0 [23968/42000(57.04%)]	Loss: 0.173501
Epoch: 0 [24000/42000(57.12%)]	Loss: 0.222131
Epoch: 0 [24032/42000(57.2%)]	Loss: 0.220271
Epoch: 0 [24064/42000(57.27%)]	Loss: 0.125609
Epoch: 0 [24096/42000(57.35%)]	Loss: 0.419169
Epoch: 0 [24128/42000(57.43%)]	Loss: 0.28556
Epoch: 0 [24160/42000(57.5%)]	Loss: 0.126376
Epoch: 0 [24192/42000(57.58%)]	Loss: 0.131498
Epoch: 0 [24224/42000(57.65%)]	Loss: 0.390309
Epoch: 0 [24256/42000(57.73%)]	Loss: 0.253623
Epoch: 0 [24288/42000(57.81%)]	Loss: 0.398654
Epoch: 0 [24320/42000(57.88%)]	Loss: 0.19371
Epoch: 0 [24352/42000(57.96%)]	Loss: 0.076876
Epoch: 0 [24384/42000(58.04%)]	Loss: 0.155076
Epoch: 0 [24416/42000(58.11%)]	Loss: 0.117313
Epoch: 0 [24448/42000(58.19%)]	Loss: 0.424782
Epoch: 0 [24480/42000(58.26%)]	Loss: 0.119206
Epoch: 0 [24512/42000(58.34%)]	Loss: 0.14403
Epoch: 0 [24544/42000(58.42%)]	Loss: 0.085365
Epoch: 0 [24576/42000(58.49%)]	Loss: 0.195817
Epoch: 0 [24608/42000(58.57%)]	Loss: 0.

...

Epoch: 9 [15936/42000(37.93%)]	Loss: 0.070364
Epoch: 9 [15968/42000(38.0%)]	Loss: 0.008855
Epoch: 9 [16000/42000(38.08%)]	Loss: 0.000555
Epoch: 9 [16032/42000(38.16%)]	Loss: 0.000669
Epoch: 9 [16064/42000(38.23%)]	Loss: 0.432033
Epoch: 9 [16096/42000(38.31%)]	Loss: 0.027682
Epoch: 9 [16128/42000(38.39%)]	Loss: 0.072894
Epoch: 9 [16160/42000(38.46%)]	Loss: 0.001033
Epoch: 9 [16192/42000(38.54%)]	Loss: 0.190806
Epoch: 9 [16224/42000(38.61%)]	Loss: 0.372937
Epoch: 9 [16256/42000(38.69%)]	Loss: 0.240336
Epoch: 9 [16288/42000(38.77%)]	Loss: 0.000113
Epoch: 9 [16320/42000(38.84%)]	Loss: 0.116789
Epoch: 9 [16352/42000(38.92%)]	Loss: 0.189294
Epoch: 9 [16384/42000(38.99%)]	Loss: 0.110679
Epoch: 9 [16416/42000(39.07%)]	Loss: 0.008934
Epoch: 9 [16448/42000(39.15%)]	Loss: 0.157394
Epoch: 9 [16480/42000(39.22%)]	Loss: 0.105181
Epoch: 9 [16512/42000(39.3%)]	Loss: 0.003391
Epoch: 9 [16544/42000(39.38%)]	Loss: 0.616649
Epoch: 9 [16576/42000(39.45%)]	Loss: 0.00509
Epoch: 9 [16608/42000(39.53%)]	Loss: 

Epoch: 9 [21792/42000(51.87%)]	Loss: 0.163824
Epoch: 9 [21824/42000(51.94%)]	Loss: 0.000405
Epoch: 9 [21856/42000(52.02%)]	Loss: 0.017026
Epoch: 9 [21888/42000(52.09%)]	Loss: 0.132293
Epoch: 9 [21920/42000(52.17%)]	Loss: 0.020244
Epoch: 9 [21952/42000(52.25%)]	Loss: 0.033602
Epoch: 9 [21984/42000(52.32%)]	Loss: 0.006239
Epoch: 9 [22016/42000(52.4%)]	Loss: 0.080927
Epoch: 9 [22048/42000(52.48%)]	Loss: 0.000371
Epoch: 9 [22080/42000(52.55%)]	Loss: 0.061525
Epoch: 9 [22112/42000(52.63%)]	Loss: 0.119188
Epoch: 9 [22144/42000(52.7%)]	Loss: 0.034702
Epoch: 9 [22176/42000(52.78%)]	Loss: 0.034541
Epoch: 9 [22208/42000(52.86%)]	Loss: 0.147937
Epoch: 9 [22240/42000(52.93%)]	Loss: 0.020847
Epoch: 9 [22272/42000(53.01%)]	Loss: 0.00187
Epoch: 9 [22304/42000(53.08%)]	Loss: 0.834454
Epoch: 9 [22336/42000(53.16%)]	Loss: 0.011334
Epoch: 9 [22368/42000(53.24%)]	Loss: 0.086778
Epoch: 9 [22400/42000(53.31%)]	Loss: 0.003879
Epoch: 9 [22432/42000(53.39%)]	Loss: 0.061544
Epoch: 9 [22464/42000(53.47%)]	Loss: 

Epoch: 9 [27680/42000(65.88%)]	Loss: 0.025928
Epoch: 9 [27712/42000(65.96%)]	Loss: 0.00728
Epoch: 9 [27744/42000(66.03%)]	Loss: 0.113674
Epoch: 9 [27776/42000(66.11%)]	Loss: 0.0753
Epoch: 9 [27808/42000(66.18%)]	Loss: 0.01093
Epoch: 9 [27840/42000(66.26%)]	Loss: 0.073917
Epoch: 9 [27872/42000(66.34%)]	Loss: 0.000974
Epoch: 9 [27904/42000(66.41%)]	Loss: 0.04969
Epoch: 9 [27936/42000(66.49%)]	Loss: 0.053141
Epoch: 9 [27968/42000(66.57%)]	Loss: 0.156367
Epoch: 9 [28000/42000(66.64%)]	Loss: 0.082395
Epoch: 9 [28032/42000(66.72%)]	Loss: 0.000284
Epoch: 9 [28064/42000(66.79%)]	Loss: 0.001149
Epoch: 9 [28096/42000(66.87%)]	Loss: 0.001241
Epoch: 9 [28128/42000(66.95%)]	Loss: 0.042462
Epoch: 9 [28160/42000(67.02%)]	Loss: 0.012085
Epoch: 9 [28192/42000(67.1%)]	Loss: 0.024189
Epoch: 9 [28224/42000(67.17%)]	Loss: 0.01797
Epoch: 9 [28256/42000(67.25%)]	Loss: 0.001441
Epoch: 9 [28288/42000(67.33%)]	Loss: 2e-05
Epoch: 9 [28320/42000(67.4%)]	Loss: 0.000691
Epoch: 9 [28352/42000(67.48%)]	Loss: 0.023044

Epoch: 9 [33568/42000(79.89%)]	Loss: 0.02804
Epoch: 9 [33600/42000(79.97%)]	Loss: 0.085652
Epoch: 9 [33632/42000(80.05%)]	Loss: 0.067386
Epoch: 9 [33664/42000(80.12%)]	Loss: 0.002473
Epoch: 9 [33696/42000(80.2%)]	Loss: 0.005563
Epoch: 9 [33728/42000(80.27%)]	Loss: 0.01223
Epoch: 9 [33760/42000(80.35%)]	Loss: 0.001537
Epoch: 9 [33792/42000(80.43%)]	Loss: 0.004533
Epoch: 9 [33824/42000(80.5%)]	Loss: 0.016154
Epoch: 9 [33856/42000(80.58%)]	Loss: 0.049423
Epoch: 9 [33888/42000(80.65%)]	Loss: 0.00152
Epoch: 9 [33920/42000(80.73%)]	Loss: 0.062072
Epoch: 9 [33952/42000(80.81%)]	Loss: 0.502851
Epoch: 9 [33984/42000(80.88%)]	Loss: 0.007095
Epoch: 9 [34016/42000(80.96%)]	Loss: 0.330175
Epoch: 9 [34048/42000(81.04%)]	Loss: 0.077529
Epoch: 9 [34080/42000(81.11%)]	Loss: 0.053042
Epoch: 9 [34112/42000(81.19%)]	Loss: 0.039999
Epoch: 9 [34144/42000(81.26%)]	Loss: 0.051573
Epoch: 9 [34176/42000(81.34%)]	Loss: 0.002811
Epoch: 9 [34208/42000(81.42%)]	Loss: 0.001708
Epoch: 9 [34240/42000(81.49%)]	Loss: 0.

Epoch: 9 [39456/42000(93.91%)]	Loss: 0.118491
Epoch: 9 [39488/42000(93.98%)]	Loss: 0.019273
Epoch: 9 [39520/42000(94.06%)]	Loss: 0.432232
Epoch: 9 [39552/42000(94.14%)]	Loss: 0.211911
Epoch: 9 [39584/42000(94.21%)]	Loss: 0.02082
Epoch: 9 [39616/42000(94.29%)]	Loss: 0.006776
Epoch: 9 [39648/42000(94.36%)]	Loss: 0.127704
Epoch: 9 [39680/42000(94.44%)]	Loss: 0.00071
Epoch: 9 [39712/42000(94.52%)]	Loss: 0.117121
Epoch: 9 [39744/42000(94.59%)]	Loss: 0.05754
Epoch: 9 [39776/42000(94.67%)]	Loss: 0.066272
Epoch: 9 [39808/42000(94.74%)]	Loss: 0.169467
Epoch: 9 [39840/42000(94.82%)]	Loss: 0.187399
Epoch: 9 [39872/42000(94.9%)]	Loss: 0.146129
Epoch: 9 [39904/42000(94.97%)]	Loss: 0.018876
Epoch: 9 [39936/42000(95.05%)]	Loss: 0.064153
Epoch: 9 [39968/42000(95.13%)]	Loss: 0.038029
Epoch: 9 [40000/42000(95.2%)]	Loss: 0.087335
Epoch: 9 [40032/42000(95.28%)]	Loss: 0.000964
Epoch: 9 [40064/42000(95.35%)]	Loss: 0.126815
Epoch: 9 [40096/42000(95.43%)]	Loss: 0.030601
Epoch: 9 [40128/42000(95.51%)]	Loss: 0.

## Final evaluation over the test set

In [185]:
    test_loss = 0
    correct = 0
    with torch.no_grad():
        for data, target in test_loader:
            output = basic_cnn(data)
            test_loss += criteria(output, target).item() # sum up batch loss
            pred = output.max(1, keepdim=True)[1] # get the index of the max log-probability
            correct += pred.eq(target.view_as(pred)).sum().item()

    test_loss /= len(test_loader.dataset)
    print('\n'
          f'Test set: Average loss: {round(test_loss, 6)}, '
          f'Accuracy: {correct}/{len(test_loader.dataset)} '
          f'({round(100. * correct / len(test_loader.dataset), 3)}%)\n')


Test set: Average loss: 0.003109, Accuracy: 9752/10000 (97.52%)

